# Getting user's location

In [1]:
# Importing required modules
import sys
from ui.finder_location_gui import UserLocation

# Creating application
user_location_gui = UserLocation()

# Running GUI and getting user's response
user_location_list = user_location_gui.run_GUI()

# Deleting application
del user_location_gui

# Checking user's response (debug)
if len(user_location_list) == 0:
    print('No input data')
else:
    print(user_location_list)

QSettings::value: Empty key passed
QSettings::value: Empty key passed


['Санкт-Петербург, Кронверский проспект, 49']


# Checking the type of input location and converting to a list of floats

In [2]:
# Importing required modules
from clients.yandex_geocoder.yandex_geocoder_client import YandexGeocoderClient

# Checking type of response
if len(user_location_list) == 1:
    location_convert = YandexGeocoderClient('9b6abe72-c629-42d6-aec0-37393d031787')
    user_location_list_temp = location_convert.get_location_by_address(user_location_list)
    user_location_list = [user_location_list_temp[1], user_location_list_temp[0]]
else:
    user_location_list = [float(i) for i in user_location_list]

# Printing result (debug)   
print(user_location_list)

[59.956165, 30.309382]


# Getting current time and weather of the user

In [3]:
# Importing required modules
from datetime import datetime
from clients.openweather.openweather_client import OpenWeatherClient

# Getting current date and time
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Getting current weather
weather_client = OpenWeatherClient('5fd3acb9e271365266e92fce70c407d3')
current_weather = weather_client.get_current_weather(user_location_list).clouds.all

# Displaying current date/time and weather (debug)
print('Current date and time: ' + current_datetime)
print('Current weather: ' + str(current_weather))

Current date and time: 2021-10-24 23:03:18
Current weather: 0


# Setting deltas for requested data

In [4]:
hour_delta = 10
month_delta = 1
latitude_delta = 0.5
longitude_delta = 0.5
weather_delta = 30

# Creating searching intervals for database request

In [5]:
# Getting current hour and month from date/time
current_hour = int(current_datetime[11:13])
current_month = int(current_datetime[5:7])

# Calculating left and right borders of tolerance interval
hour_left = (current_hour - hour_delta + 24) % 24; hour_right = (current_hour + hour_delta + 24) % 24
month_left = (current_month - month_delta) % 12; month_right = (current_month + month_delta) % 12
latitude_left = user_location_list[0] - latitude_delta; latitude_right = user_location_list[0] + latitude_delta
longitude_left = user_location_list[1] - longitude_delta; longitude_right = user_location_list[1] + longitude_delta
weather_left = current_weather - weather_delta; weather_right = current_weather + weather_delta

# Getting entries id from database

In [6]:
# Importing required modules
from database.database_sqlite import ImageDatabase

db = ImageDatabase('database/photos.db')
entry_id_list = db.get_entry_id_by_parameters(hour_left, hour_right, 
                                              month_left, month_right,
                                              latitude_left, latitude_right,
                                              longitude_left, longitude_right,
                                              weather_left, weather_right)

# Displaying list of obtained entries (debug)
print(entry_id_list)

SELECT PhotoID FROM Photos WHERE TRUE AND ((cast(strftime('%H', time(PhotoDateTime)) as INTEGER) >= 13) OR (cast(strftime('%H', time(PhotoDateTime)) as INTEGER) <= 9)) AND (cast(strftime('%m', date(PhotoDateTime)) as INTEGER) BETWEEN 9 AND 11) AND (LocationLatitude BETWEEN 59.456165 AND 60.456165) AND (LocationLongitude BETWEEN 29.809382 AND 30.809382) AND (Weather BETWEEN -30 AND 30)
[1]


# Getting images location by their entry id

In [7]:
# Creating list for images locations storage
images_location_list = []

# Getting location for every entry
for entry_id in entry_id_list:
    image_location = list(db.get_location_by_entry_id(entry_id))
    images_location_list.append(image_location)
    
# Displaying obtained locations (debug)
print(images_location_list)

[[60.0, 30.0]]


# Getting images colors by their entry id

In [8]:
# Creating list for images colors storage
images_colors_list = []

# Getting colors for every entry
for entry_id in entry_id_list:
    image_colors = list(db.get_colors_by_entry_id(entry_id))
    images_colors_list.append(image_colors)
    
# Displaying obtained colors (debug)
print(images_colors_list)

[[0.1, 0.1, 0.1, 10]]


# Displaying colors on the map